In [28]:
df = spark.read.format("jdbc") \
                .option("url", "jdbc:mysql://"+os.environ["DB_CONNECTION_URL"]+"/?serverTimezone=UTC") \
                .option("driver", "com.mysql.jdbc.Driver") \
                .option("dbtable", "demodb.customers") \
                .option("user", os.environ["DB_USER"]).option("password", os.environ["DB_PASSWORD"]) \
                .load()

In [29]:
df.createOrReplaceTempView('df')

In [30]:
graduated_and_predictions = sql.sql(sqlQuery='SELECT Date_record,Loan_ID,\
                                predict FROM df WHERE Education_Not_Graduate == 0')

In [31]:
graduated_and_predictions.show()

+-----------+---------+-------+
|Date_record|  Loan_ID|predict|
+-----------+---------+-------+
| 2019-12-31|LPP003672|      1|
| 2019-12-31|LPP003674|      0|
| 2019-12-31|LPP003675|      1|
| 2019-12-31|LPP003677|      1|
| 2019-12-31|LPP003679|      1|
| 2019-12-31|LPP003680|      0|
| 2019-12-31|LPP003681|      0|
| 2019-12-31|LPP003682|      1|
| 2019-12-31|LPP003683|      0|
| 2019-12-31|LPP003685|      1|
| 2019-12-31|LPP003686|      1|
| 2019-12-31|LPP003687|      0|
| 2019-12-31|LPP003688|      1|
| 2019-12-31|LPP003689|      1|
| 2019-12-31|LPP003690|      1|
| 2019-12-31|LPP003692|      1|
| 2019-12-31|LPP003693|      1|
| 2019-12-31|LPP003694|      0|
| 2019-12-31|LPP003695|      1|
| 2019-12-31|LPP003696|      1|
+-----------+---------+-------+
only showing top 20 rows



In [32]:
graduated_and_predictions.write.mode('append').format("jdbc") \
                            .option("url", "jdbc:mysql://"+os.environ["DB_CONNECTION_URL"]+"/?serverTimezone=UTC") \
                            .option("dbtable", "demodb.graduated") \
                            .option("user", os.environ["DB_USER"]).option("password", os.environ["DB_PASSWORD"]) \
                            .save()

DAM: in patched DataFrame.write, marking DataFrame as cached
DAM: in patched DataFrame.write, returning result


# Looking at logs

In [33]:
logs = read_pyspark_logs()

In [34]:
logs

[{'action': 'add_entity',
  'entity': 'BATCH_ENTITY_REPORT',
  'generatedEntityGUID': 'a02101ea-957b-4c58-a193-27670be2e6d0',
  'schemaVersion': '0.1',
  'jsonPayload': {'projects': [{'timestamp': 1616628349031,
     'entity': {'pk': {'name': 'AI Observability Meetup'}}}],
   'processes': [{'timestamp': 1616628349049,
     'entity': {'pk': {'qualifiedName': 'notebooks/AI Observability PySpark.ipynb'}}}],
   'processRuns': [{'timestamp': 1616628349237,
     'entity': {'pk': {'processRef': {'byGUID': 'k-98fe8f5eb931e62ea00d1ce53797b62b7741cf02277c58828baee6afa27271a5'},
       'qualifiedName': 'local-1616628210771'},
      'launchedByUserRef': {'byGUID': 'k-bea8aa69af5be716c580fa2d2d63b0964bdb89f74d484319728aeb16add844e9'},
      'executedCodeVersionRef': {'byGUID': 'k-578c1fc62671c14a89b6e061f1a681fa1bd563565d0e05ca0451461b5c59f1c2'},
      'environment': 'Lab',
      'projectsRefs': [{'byGUID': 'k-5ea877c962287021c2319b333362fac7f507c948a81c9679c054a00d685e4103'}]}}],
   'processLineag

In [35]:
logs[0]['jsonPayload']

{'projects': [{'timestamp': 1616628349031,
   'entity': {'pk': {'name': 'AI Observability Meetup'}}}],
 'processes': [{'timestamp': 1616628349049,
   'entity': {'pk': {'qualifiedName': 'notebooks/AI Observability PySpark.ipynb'}}}],
 'processRuns': [{'timestamp': 1616628349237,
   'entity': {'pk': {'processRef': {'byGUID': 'k-98fe8f5eb931e62ea00d1ce53797b62b7741cf02277c58828baee6afa27271a5'},
     'qualifiedName': 'local-1616628210771'},
    'launchedByUserRef': {'byGUID': 'k-bea8aa69af5be716c580fa2d2d63b0964bdb89f74d484319728aeb16add844e9'},
    'executedCodeVersionRef': {'byGUID': 'k-578c1fc62671c14a89b6e061f1a681fa1bd563565d0e05ca0451461b5c59f1c2'},
    'environment': 'Lab',
    'projectsRefs': [{'byGUID': 'k-5ea877c962287021c2319b333362fac7f507c948a81c9679c054a00d685e4103'}]}}],
 'processLineages': [{'timestamp': 1616628349296,
   'entity': {'name': 'Generate `schema::mysql-kensu-demo.usnek.com:3306/demodb.graduated` from `schema::mysql-kensu-demo.usnek.com:3306/demodb.customers`',

In [36]:
logs[0]['jsonPayload']["processes"]

[{'timestamp': 1616628349049,
  'entity': {'pk': {'qualifiedName': 'notebooks/AI Observability PySpark.ipynb'}}}]

In [37]:
logs[0]['jsonPayload']["codeBases"]

[{'timestamp': 1616628349024,
  'entity': {'pk': {'location': 'git@github.com/kensu/python-examples.git'}}}]

In [38]:
logs[0]['jsonPayload']["users"]

[{'timestamp': 1616628349025, 'entity': {'pk': {'name': 'andy'}}},
 {'timestamp': 1616628349023, 'entity': {'pk': {'name': 'andy'}}}]

In [39]:
logs[0]['jsonPayload']["dataSources"]

[{'timestamp': 1616628349229,
  'entity': {'name': 'mysql-kensu-demo.usnek.com:3306/demodb.graduated',
   'format': 'mysql',
   'categories': ['logical::demodb.graduated'],
   'pk': {'location': 'mysql://mysql-kensu-demo.usnek.com:3306/demodb.graduated',
    'physicalLocationRef': {'byGUID': 'k-d2f40e99e5dd4c9fc9c634b15a7fb03073191c0158e52a572769df8c05f59b7b'}}}},
 {'timestamp': 1616628349225,
  'entity': {'name': 'mysql-kensu-demo.usnek.com:3306/demodb.customers',
   'format': 'mysql',
   'categories': ['logical::demodb.customers'],
   'pk': {'location': 'mysql://mysql-kensu-demo.usnek.com:3306/demodb.customers',
    'physicalLocationRef': {'byGUID': 'k-d2f40e99e5dd4c9fc9c634b15a7fb03073191c0158e52a572769df8c05f59b7b'}}}}]

In [40]:
logs[0]['jsonPayload']["schemas"][0]

{'timestamp': 1616628349232,
 'entity': {'name': 'schema::mysql-kensu-demo.usnek.com:3306/demodb.graduated',
  'pk': {'dataSourceRef': {'byGUID': 'k-981bf9c50b02272ac666ab49edf81336ca6982a75e0634c9b3bcaf17e404ba05'},
   'fields': [{'name': 'Date_record', 'fieldType': 'date', 'nullable': True},
    {'name': 'Loan_ID', 'fieldType': 'string', 'nullable': True},
    {'name': 'predict', 'fieldType': 'integer', 'nullable': True}]}}}

In [41]:
logs[0]['jsonPayload']["processLineages"]

[{'timestamp': 1616628349296,
  'entity': {'name': 'Generate `schema::mysql-kensu-demo.usnek.com:3306/demodb.graduated` from `schema::mysql-kensu-demo.usnek.com:3306/demodb.customers`',
   'operationLogic': 'APPEND',
   'pk': {'processRef': {'byGUID': 'k-98fe8f5eb931e62ea00d1ce53797b62b7741cf02277c58828baee6afa27271a5'},
    'dataFlow': [{'fromSchemaRef': {'byGUID': 'k-2c737ac3b26450791044e5ef1af4e94e6475bba000492c7c9a21fd5cb147f170'},
      'toSchemaRef': {'byGUID': 'k-3ca0a8edcf98a10849dc3dcbc7cae05e87ed0ad589677edec88378dd82840237'},
      'columnDataDependencies': {'Date_record': ['Date_record'],
       'predict': ['predict'],
       'Loan_ID': ['Loan_ID']},
      'columnControlDependencies': {'Date_record': ['Education_Not_Graduate'],
       'predict': ['Education_Not_Graduate'],
       'Loan_ID': ['Education_Not_Graduate']}}]}}}]

In [42]:
logs[0]['jsonPayload']["processLineages"][0]["entity"]["pk"]["dataFlow"][0]["columnDataDependencies"]

{'Date_record': ['Date_record'],
 'predict': ['predict'],
 'Loan_ID': ['Loan_ID']}

In [43]:
logs[0]['jsonPayload']["processLineages"][0]["entity"]["pk"]["dataFlow"][0]["columnControlDependencies"]

{'Date_record': ['Education_Not_Graduate'],
 'predict': ['Education_Not_Graduate'],
 'Loan_ID': ['Education_Not_Graduate']}

In [44]:
logs[1]['jsonPayload']["dataStats"]

[{'timestamp': 1616628353963,
  'entity': {'pk': {'schemaRef': {'byGUID': 'k-3ca0a8edcf98a10849dc3dcbc7cae05e87ed0ad589677edec88378dd82840237'},
    'lineageRunRef': {'byGUID': 'k-f6009204ac214429d3e7509a1f158ff79c4b68bb18c75dd45a53480d6623de77'}},
   'stats': {'Loan_ID.nullrows': 0.0,
    'predict.25%': 1.0,
    'predict.median': 1.0,
    'predict.75%': 1.0,
    'predict.stddev': 0.4218455440114891,
    'predict.nrows': 3987.0,
    'nrows': 3987.0,
    'predict.min': 0.0,
    'predict.mean': 0.7684976172560822,
    'Loan_ID.nrows': 3987.0,
    'predict.nullrows': 0.0,
    'predict.max': 1.0}}}]

# Strategy

* Event interception: 
    * https://github.com/kensuio/dam-spline-persistence/blob/33e1b1d50cae82bd996c2bbc46927bdeace7b1ee/src/main/scala/io/kensu/third/integration/spline/utils/spline/DamQueryExecutionEventHandler.scala#L45
    * https://github.com/kensuio/dam-spline-persistence/blob/develop/src/main/scala/DAMDataLineageWriter.scala#L250
* ASM: https://github.com/kensuio/dam-spline-persistence/blob/develop/src/main/scala/dam-spark-mlib-interceptor/DamModelPublisher.scala#L62